In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import datetime

In [2]:
train = pd.read_csv('../data/weather1_education_train.csv')
train['timestamp'] = train['timestamp'].apply(parser.parse)
train.head()

,Unnamed: 0,area,building_name,electricity,primary_space_usage,timestamp,TemperatureC,month,year,date,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,0,2777.0,PrimClass_Jacqueline,5.0,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
1,1,2777.0,PrimClass_Jacqueline,5.2,PrimClass,2014-12-01 01:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
2,2,2777.0,PrimClass_Jacqueline,5.2,PrimClass,2014-12-01 02:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
3,3,2777.0,PrimClass_Jacqueline,5.5,PrimClass,2014-12-01 03:00:00,6.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
4,4,2777.0,PrimClass_Jacqueline,6.0,PrimClass,2014-12-01 04:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0


In [3]:
test = pd.read_csv('../data/weather1_education_test.csv')
test['timestamp'] = test['timestamp'].apply(parser.parse)

In [4]:
three_month = 2159 #from 12/01 0:00 to 1/31 23:00
pred_len = 6598 #length of prediction, 8757 is the length of 1 year-long observation

In [5]:
#nine_month = 6574 #from 12/01 0:00 to 8/31 23:00
#pred_len = 8757 - three_month #length of prediction, 8757 is the length of 1 year-long observation

In [6]:
def build_model_1(train_len, pred_len):
    inputs = layers.Input(shape=(1, three_month)) #temporal data: electricity
    x = layers.LSTM(units=pred_len, input_shape=(1, three_month), activation='relu')(inputs)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [7]:
def make_input_1(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 1, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [8]:
arr = np.array([1, 2])
1/arr

array([1. , 0.5])

In [9]:
# from tensorflow.keras import backend as K

# def dev_reg(weight_matrix):
#     n, m = weight_matrix.shape
#     avg = K.mean(weight_matrix)
#     return 100*K.sum(K.abs(1/(weight_matrix + 0.00001)))

In [10]:
def build_model_2(train_len, pred_len):
    inputs = layers.Input(shape=(1, train_len)) #temporal data: electricity
    x = layers.LSTM(units=128, input_shape=(1, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [11]:
def make_input_2(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 1, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [12]:
def build_model_3(train_len, pred_len):
    inputs = layers.Input(shape=(2, train_len)) #temporal data: electricity
    x = layers.LSTM(units=50, input_shape=(2, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [13]:
def make_input_3(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity', 'TemperatureC']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 2, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [14]:
def build_model_4(train_len, pred_len):
    inputs = layers.Input(shape=(9, train_len)) #temporal data: electricity
    x = layers.LSTM(units=64, input_shape=(9, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [15]:
def make_input_4(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity', 'TemperatureC',
                                              'wkday_0', 'wkday_1', 'wkday_2', 'wkday_3', 'wkday_4', 'wkday_5', 'wkday_6']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 9, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [16]:
def build_model_5(train_len, pred_len):
    inputs = layers.Input(shape=(21, train_len)) #temporal data: electricity
    x = layers.LSTM(units=50, input_shape=(21, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [17]:
def make_input_5(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity', 'TemperatureC',
                                              'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
                                              'wkday_0', 'wkday_1', 'wkday_2', 'wkday_3', 'wkday_4', 'wkday_5', 'wkday_6']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 21, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [18]:
def build_model_6(train_len, pred_len):
    inputs = layers.Input(shape=(1, train_len)) #temporal data: electricity
    x = layers.LSTM(units=128, input_shape=(1, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    x = layers.Reshape((pred_len, 1))(x)
    x = layers.Conv1D(filters=1, kernel_size=(5), padding='same')(x)
    x = layers.Reshape((pred_len,))(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [19]:
def make_input_6(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 1, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [20]:
def build_model_7(train_len, pred_len):
    inputs = layers.Input(shape=(8, train_len)) #temporal data: electricity
    x = layers.LSTM(units=64, input_shape=(8, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    x = layers.Reshape((pred_len, 1))(x)
    x = layers.Conv1D(filters=1, kernel_size=(5), padding='same')(x)
    x = layers.Reshape((pred_len,))(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [21]:
def make_input_7(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity', 'TemperatureC',
                                              'wkday_1', 'wkday_2', 'wkday_3', 'wkday_4', 'wkday_5', 'wkday_6']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 8, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [59]:
def build_model_8(train_len, pred_len):
    inputs = layers.Input(shape=(7, train_len)) #temporal data: electricity
    x = layers.LSTM(units=128, input_shape=(7, train_len), activation='relu')(inputs)
    x = layers.Dense(pred_len, activation='relu')(x)
    x = layers.Reshape((pred_len, 1))(x)
    x = layers.Conv1D(filters=10, kernel_size=(5), padding='same')(x)
    x = layers.Conv1D(filters=1, kernel_size=1, padding='same')(x)
    x = layers.Reshape((pred_len,))(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
    return model

In [60]:
def make_input_8(df, date, num_buildings):
    '''
    split input dataframe into fitting set and evaluation set by the input date
    '''
    X = np.array(df[df['timestamp'] <= date][['electricity',
                                              'wkday_1', 'wkday_2', 'wkday_3', 'wkday_4', 'wkday_5', 'wkday_6']])
    Y = np.array(df[df['timestamp'] > date][['electricity']])
    len_obs = int(len(X) / num_buildings)
    len_pred = int(len(Y) / num_buildings)
    X = np.reshape(X, (num_buildings, 7, len_obs))
    Y = np.reshape(Y, (num_buildings, len_pred))
    return X, Y

In [61]:
# def build_model_x(train_len, pred_len):
#     temp_inputs = layers.Input(shape=(2, three_month)) #electricity and temperature
#     categorical_inputs = layers.Input(shape=(1,)) #primary space usage
#     x = layers.LSTM(units=256, input_shape=(three_month, 2), activation='relu')(temp_inputs)
#     z = layers.Concatenate(axis=-1)([x, categorical_inputs])
#     z = layers.Dense(pred_len, activation='relu')(z)
#     model = keras.models.Model(inputs=[temp_inputs, categorical_inputs], outputs=z)
#     model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse']) 
#     return model

In [62]:
model = build_model_8(three_month, pred_len)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 7, 2159)]         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               1171456   
_________________________________________________________________
dense_4 (Dense)              (None, 6598)              851142    
_________________________________________________________________
reshape_7 (Reshape)          (None, 6598, 1)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 6598, 10)          60        
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 6598, 1)           11        
_________________________________________________________________
reshape_8 (Reshape)          (None, 6598)              0   

In [63]:
end = datetime(2015, 2, 28, 23)
#X_train, X_train_PSU, Y_train = make_input_3(train, end, 50)
#X_test, X_test_PSU, Y_test = make_input_3(test, end, 19)
X_train, Y_train = make_input_8(train, end, 50)
X_test, Y_test = make_input_8(test, end, 19)

In [64]:
#model.fit([X_train, X_train_PSU], Y_train, epochs=1000, batch_size=10)
model.fit(X_train, Y_train, epochs=1000, batch_size=10)

Epoch 1/1000
50/50 [==============================] - 2s 41ms/sample - loss: 1428.6905 - mean_absolute_error: 20.9272 - mean_squared_error: 1428.6904
Epoch 2/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1418.7347 - mean_absolute_error: 20.8014 - mean_squared_error: 1418.7347
Epoch 3/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1417.5368 - mean_absolute_error: 20.7729 - mean_squared_error: 1417.5369
Epoch 4/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1416.4618 - mean_absolute_error: 20.7453 - mean_squared_error: 1416.4617
Epoch 5/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1415.2376 - mean_absolute_error: 20.7173 - mean_squared_error: 1415.2375
Epoch 6/1000
50/50 [==============================] - 1s 13ms/sample - loss: 1414.1274 - mean_absolute_error: 20.6890 - mean_squared_error: 1414.1273
Epoch 7/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1412.8823 - mean_absolu

50/50 [==============================] - 1s 11ms/sample - loss: 1322.6684 - mean_absolute_error: 18.6143 - mean_squared_error: 1322.6685
Epoch 56/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1320.2155 - mean_absolute_error: 18.5637 - mean_squared_error: 1320.2155
Epoch 57/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1317.7059 - mean_absolute_error: 18.5106 - mean_squared_error: 1317.7059
Epoch 58/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1314.8769 - mean_absolute_error: 18.4584 - mean_squared_error: 1314.8768
Epoch 59/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1312.1442 - mean_absolute_error: 18.4060 - mean_squared_error: 1312.1442
Epoch 60/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1309.5039 - mean_absolute_error: 18.3568 - mean_squared_error: 1309.5039
Epoch 61/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1307.0166 - mean_absolute_erro

50/50 [==============================] - 1s 11ms/sample - loss: 1169.1835 - mean_absolute_error: 17.2648 - mean_squared_error: 1169.1835
Epoch 110/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1166.3861 - mean_absolute_error: 17.2712 - mean_squared_error: 1166.3862
Epoch 111/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1163.6200 - mean_absolute_error: 17.2805 - mean_squared_error: 1163.6201
Epoch 112/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1160.9882 - mean_absolute_error: 17.2859 - mean_squared_error: 1160.9882
Epoch 113/1000
50/50 [==============================] - 1s 13ms/sample - loss: 1158.2424 - mean_absolute_error: 17.2943 - mean_squared_error: 1158.2424
Epoch 114/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1155.5883 - mean_absolute_error: 17.3057 - mean_squared_error: 1155.5883
Epoch 115/1000
50/50 [==============================] - 1s 11ms/sample - loss: 1152.8411 - mean_absolut

Epoch 163/1000
50/50 [==============================] - 1s 11ms/sample - loss: 1050.0176 - mean_absolute_error: 18.0104 - mean_squared_error: 1050.0175
Epoch 164/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1048.2209 - mean_absolute_error: 18.0247 - mean_squared_error: 1048.2208
Epoch 165/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1047.3651 - mean_absolute_error: 18.0507 - mean_squared_error: 1047.3650
Epoch 166/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1045.7220 - mean_absolute_error: 18.0715 - mean_squared_error: 1045.7219
Epoch 167/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1043.9506 - mean_absolute_error: 18.0847 - mean_squared_error: 1043.9506
Epoch 168/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1042.5663 - mean_absolute_error: 18.1004 - mean_squared_error: 1042.5662
Epoch 169/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1041.3870

50/50 [==============================] - 1s 11ms/sample - loss: 1000.5579 - mean_absolute_error: 18.9981 - mean_squared_error: 1000.5580
Epoch 217/1000
50/50 [==============================] - 1s 12ms/sample - loss: 1000.1575 - mean_absolute_error: 19.0159 - mean_squared_error: 1000.1575
Epoch 218/1000
50/50 [==============================] - 1s 12ms/sample - loss: 999.5057 - mean_absolute_error: 19.0295 - mean_squared_error: 999.5058
Epoch 219/1000
50/50 [==============================] - 1s 12ms/sample - loss: 999.1389 - mean_absolute_error: 19.0453 - mean_squared_error: 999.1389
Epoch 220/1000
50/50 [==============================] - 1s 12ms/sample - loss: 998.8768 - mean_absolute_error: 19.0657 - mean_squared_error: 998.8769
Epoch 221/1000
50/50 [==============================] - 1s 12ms/sample - loss: 998.4251 - mean_absolute_error: 19.0834 - mean_squared_error: 998.4250
Epoch 222/1000
50/50 [==============================] - 1s 12ms/sample - loss: 997.8705 - mean_absolute_error: 

50/50 [==============================] - 1s 12ms/sample - loss: 988.2144 - mean_absolute_error: 19.7213 - mean_squared_error: 988.2143
Epoch 271/1000
50/50 [==============================] - 1s 12ms/sample - loss: 988.0893 - mean_absolute_error: 19.7289 - mean_squared_error: 988.0893
Epoch 272/1000
50/50 [==============================] - 1s 11ms/sample - loss: 988.0356 - mean_absolute_error: 19.7359 - mean_squared_error: 988.0356
Epoch 273/1000
50/50 [==============================] - 1s 12ms/sample - loss: 987.9544 - mean_absolute_error: 19.7442 - mean_squared_error: 987.9545
Epoch 274/1000
50/50 [==============================] - 1s 12ms/sample - loss: 987.9339 - mean_absolute_error: 19.7534 - mean_squared_error: 987.9340
Epoch 275/1000
50/50 [==============================] - 1s 12ms/sample - loss: 987.8152 - mean_absolute_error: 19.7612 - mean_squared_error: 987.8152
Epoch 276/1000
50/50 [==============================] - 1s 12ms/sample - loss: 987.7766 - mean_absolute_error: 19.7

50/50 [==============================] - 1s 13ms/sample - loss: 986.4704 - mean_absolute_error: 20.0516 - mean_squared_error: 986.4704
Epoch 325/1000
50/50 [==============================] - 1s 13ms/sample - loss: 986.3271 - mean_absolute_error: 20.0569 - mean_squared_error: 986.3270
Epoch 326/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.3737 - mean_absolute_error: 20.0625 - mean_squared_error: 986.3736
Epoch 327/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.2998 - mean_absolute_error: 20.0647 - mean_squared_error: 986.2997
Epoch 328/1000
50/50 [==============================] - 1s 13ms/sample - loss: 986.3267 - mean_absolute_error: 20.0647 - mean_squared_error: 986.3267
Epoch 329/1000
50/50 [==============================] - 1s 13ms/sample - loss: 986.3210 - mean_absolute_error: 20.0644 - mean_squared_error: 986.3210
Epoch 330/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.2938 - mean_absolute_error: 20.0

50/50 [==============================] - 1s 11ms/sample - loss: 986.1488 - mean_absolute_error: 20.1690 - mean_squared_error: 986.1488
Epoch 379/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1390 - mean_absolute_error: 20.1688 - mean_squared_error: 986.1389
Epoch 380/1000
50/50 [==============================] - 1s 11ms/sample - loss: 986.1655 - mean_absolute_error: 20.1740 - mean_squared_error: 986.1655
Epoch 381/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1587 - mean_absolute_error: 20.1787 - mean_squared_error: 986.1588
Epoch 382/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1530 - mean_absolute_error: 20.1810 - mean_squared_error: 986.1530
Epoch 383/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1591 - mean_absolute_error: 20.1802 - mean_squared_error: 986.1591
Epoch 384/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1612 - mean_absolute_error: 20.1

50/50 [==============================] - 1s 11ms/sample - loss: 986.1210 - mean_absolute_error: 20.2072 - mean_squared_error: 986.1209
Epoch 433/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1512 - mean_absolute_error: 20.2111 - mean_squared_error: 986.1512
Epoch 434/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1508 - mean_absolute_error: 20.2129 - mean_squared_error: 986.1508
Epoch 435/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1265 - mean_absolute_error: 20.2113 - mean_squared_error: 986.1265
Epoch 436/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1144 - mean_absolute_error: 20.2104 - mean_squared_error: 986.1144
Epoch 437/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1460 - mean_absolute_error: 20.2088 - mean_squared_error: 986.1459
Epoch 438/1000
50/50 [==============================] - 1s 12ms/sample - loss: 986.1358 - mean_absolute_error: 20.2

KeyboardInterrupt: 

In [ ]:
Y_fitted = model.predict(X_train)

In [ ]:
plt.figure(figsize=(100,10))
plt.plot(Y_fitted[0], label='fitted')
plt.plot(Y_train[0], label='observed')
plt.legend();

In [ ]:
#model.evaluate([X_test, X_test_PSU], Y_test)
model.evaluate(X_test, Y_test)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(Y_pred[0], label='predicted')
plt.plot(Y_test[0], label='observed')
plt.legend();